# 01 — Data Audit (Online Retail)

"
        "This notebook performs the initial audit: schema, missing values, duplicates, basic distributions, and quality checks.
"
        "Update the `DATA_PATH` if your CSV is located elsewhere.

In [1]:
import pandas as pd
from pathlib import Path

DATA_PATH = Path('/Users/prithabera/Downloads/OnlineRetail.csv')

# Load
df = pd.read_csv(DATA_PATH, encoding='ISO-8859-1')

# Basic view
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [ ]:
# Schema and shape
print('Shape:', df.shape)
df.info()

Shape: (541909, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [3]:
#checking null values

df.isna().sum().sort_values(ascending=False)

CustomerID     135080
Description      1454
InvoiceNo           0
StockCode           0
Quantity            0
InvoiceDate         0
UnitPrice           0
Country             0
dtype: int64

In [5]:
# How big is the missing ID issue?
total_rows = len(df)
missing_rows = df['CustomerID'].isna().sum()
missing_impact = missing_rows / total_rows * 100

print(f"Missing CustomerID rows: {missing_rows} ({missing_impact:.2f}%)")

# Revenue impact of missing IDs
df['sales'] = df['Quantity'] * df['UnitPrice']
missing_rev = df.loc[df['CustomerID'].isna(), 'sales'].sum()
total_rev = df['sales'].sum()

print(f"Revenue from missing CustomerID: {missing_rev:,.2f} ({missing_rev/total_rev*100:.2f}%)")


Missing CustomerID rows: 135080 (24.93%)
Revenue from missing CustomerID: 1,447,682.12 (14.85%)


### Insight:
#### “About 25% of transactions (≈15% of revenue) are from anonymous customers. This limits our ability to run retention and personalization campaigns, because these orders are not linked to customer identities.”

### Business recommendation:
#### “Introduce stronger identity capture (account creation, email capture at checkout, loyalty signup) to increase identified revenue and enable targeted retention.”

In [8]:
# Quick checks
print('Duplicate rows:', df.duplicated().sum())
print('Unique invoices:', df['InvoiceNo'].nunique())
print('Unique customers:', df['CustomerID'].nunique())

Duplicate rows: 5268
Unique invoices: 25900
Unique customers: 4372


In [11]:
# We have enough customers (4372) and enough invoices (25900) to do customer/invoice analysis.

# We have duplicate rows (5268) → we must clean (drop or aggregate) before any modeling, otherwise our model/EDA gets biased.

In [12]:
# Look for cancellations, negative quantities, and price issues
# Invoice numbers starting with 'C' are cancellations in this dataset

mask_cancel = df['InvoiceNo'].astype(str).str.startswith('C')

summary = {
    'cancellation_rows': int(mask_cancel.sum()),
    'negative_qty_rows': int((df['Quantity'] <= 0).sum()),
    'nonpositive_price_rows': int((df['UnitPrice'] <= 0).sum()),
}
summary

{'cancellation_rows': 9288,
 'negative_qty_rows': 10624,
 'nonpositive_price_rows': 2517}

In [13]:
# Optional: basic outlier inspection
# (not removal, just visibility)

df[['Quantity', 'UnitPrice']].describe(percentiles=[.5, .9, .95, .99])

,Quantity,UnitPrice
count,541909.000000,541909.000000
mean,9.552250,4.611114
std,218.081158,96.759853
min,-80995.000000,-11062.060000
50%,3.000000,2.080000
90%,24.000000,7.950000
95%,29.000000,9.950000
99%,100.000000,18.000000
max,80995.000000,38970.000000


In [14]:
# Date range
# Convert to datetime to check date coverage

df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
print('Date min:', df['InvoiceDate'].min())
print('Date max:', df['InvoiceDate'].max())

Date min: 2010-12-01 08:26:00
Date max: 2011-12-09 12:50:00


In [16]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
